# Data Exploration

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [6]:
directory = "data/train/"
d = os.listdir(directory)
filenames = []
for f in d:
    if ".png" not in f:
        continue
    filenames.append(directory+f)

# Exploratory Visualization